In [9]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
from tabulate import tabulate

In [3]:
print(plt.style.available)

['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', 'tableau-colorblind10', '_classic_test']


In [4]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
data = pd.read_csv('aaba.us.txt',sep=',', index_col='Date', parse_dates=['Date'], date_parser=dateparse).fillna(0)

In [5]:
data

,Open,High,Low,Close,Volume,OpenInt
Date,,,,,,
1996-04-12,1.05,1.790,1.02,1.38,408720000,0
1996-04-15,1.49,1.500,1.25,1.34,79231200,0
1996-04-16,1.34,1.340,1.17,1.20,48026400,0
1996-04-17,1.18,1.180,1.03,1.12,42871200,0
1996-04-18,1.25,1.250,1.17,1.22,27352800,0
...,...,...,...,...,...,...
2017-11-06,70.27,71.960,70.27,71.71,7129828,0
2017-11-07,72.20,72.470,71.62,72.22,9161521,0
2017-11-08,72.00,72.200,71.09,71.16,7363365,0


In [14]:
print(tabulate(data.head(50), tablefmt="pipe", headers="keys"))

| Date                |   Open |   High |   Low |   Close |      Volume |   OpenInt |
|:--------------------|-------:|-------:|------:|--------:|------------:|----------:|
| 1996-04-12 00:00:00 |   1.05 |   1.79 |  1.02 |    1.38 | 4.0872e+08  |         0 |
| 1996-04-15 00:00:00 |   1.49 |   1.5  |  1.25 |    1.34 | 7.92312e+07 |         0 |
| 1996-04-16 00:00:00 |   1.34 |   1.34 |  1.17 |    1.2  | 4.80264e+07 |         0 |
| 1996-04-17 00:00:00 |   1.18 |   1.18 |  1.03 |    1.12 | 4.28712e+07 |         0 |
| 1996-04-18 00:00:00 |   1.25 |   1.25 |  1.17 |    1.22 | 2.73528e+07 |         0 |
| 1996-04-19 00:00:00 |   1.25 |   1.28 |  1.2  |    1.21 | 1.30248e+07 |         0 |
| 1996-04-22 00:00:00 |   1.21 |   1.21 |  1.15 |    1.18 | 8.112e+06   |         0 |
| 1996-04-23 00:00:00 |   1.2  |   1.21 |  1.17 |    1.17 | 4.4208e+06  |         0 |
| 1996-04-24 00:00:00 |   1.19 |   1.22 |  1.16 |    1.21 | 7.86e+06    |         0 |
| 1996-04-25 00:00:00 |   1.25 |   1.34 |  1.21 |    1